w booking seats trzeba zrobic - dziala

cost - description


In [1]:
import math
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import csv
from datetime import datetime, timedelta
from faker import Faker
from itertools import count
from collections import defaultdict
np.random.seed(47)

historia:
loty startuja z czwartej planety systemu centralnego (o gwiezdzie w [0,0,0]).
im planeta w galaktyce dalej od ziemii tym:
- wiecej pracownikow
- drozszy lot
- mozliwe ze bedzie dostepny tylko w jedna strone

Im:
- statek ma wieksza capacity (wielkosc),
- tym cena wyzsza

planety w jednej galaktyce maja zblizone wspolrzedne.

jak ktos pojechal tylko w jedna strone - to juz nie moze pojawic sie w transakcjach i bookingach.

niech salary pracownikow bedzie najwyzsze dla commander a najnizsze dla technician i niech zalezy od doswiadczenia w firmie.



planets + coordinates + galaxy

In [2]:
# warsja rozszeżona o ruch orbitalny

# Nazwy galaktyk

with open('galaxies_example_names.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    galaxies_example = [row[0] for row in reader]

# Losuj liczbę elementów między 3 a 5
num_to_select = random.randint(3, 5)

# Wybierz losowe, unikalne elementy z listy
galaxy_names = random.sample(galaxies_example, num_to_select)

# Nazwy systemów (bogowie, łacina, mitologia)

with open('systems_example_names.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    system_names  = [row[0] for row in reader]


# Nazwy planet (mitologiczne, łacińskie)
with open('planets_example_names.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # pomija nagłówek
    planet_names  = [row[0] for row in reader]

# === Inicjalizacja ===
galaxies = []
systems = []
planets = []

galaxy_coords = {}
system_coords = {}

system_id_counter = 1
planet_id_counter = 1

# === GALAXY DATA ===
for i, name in enumerate(galaxy_names, start=1):
    galaxies.append({
        "galaxy_id": i,
        "galaxy": name
    })
    galaxy_coords[i] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# === PLANETARY SYSTEM DATA ===
for galaxy in galaxies:
    galaxy_id = galaxy["galaxy_id"]
    base = galaxy_coords[galaxy_id]

    num_systems = random.randint(2, 5)

    for _ in range(num_systems):
        raw_name = random.choice(system_names)
        name = raw_name
        
        # Dla pierwszego systemu ustaw współrzędne (0, 0, 0)
        if system_id_counter == 1:
            x, y, z = 0.0, 0.0, 0.0
        else:
            x = round(base["base_x"] + random.uniform(-100, 100), 2)
            y = round(base["base_y"] + random.uniform(-100, 100), 2)
            z = round(base["base_z"] + random.uniform(-100, 100), 2)


        systems.append({
            "system_id": system_id_counter,
            "system": name,
            "central_star": f"{raw_name} Star",
            "x_coord": x,
            "y_coord": y,
            "z_coord": z,
            "galaxy_id": galaxy_id
        })

        system_coords[system_id_counter] = {"x": x, "y": y, "z": z}
        system_id_counter += 1

# === PLANET DATA ===

for system in systems:
    system_id = system["system_id"]
    base = system_coords[system_id]
    num_planets = random.randint(2, 4)

    for _ in range(num_planets):
        raw_name = random.choice(planet_names)
        name = raw_name
        ## planet_name_count[raw_name] += 1
        ## name = raw_name if planet_name_count[raw_name] == 1 else f"{raw_name}_{planet_name_count[raw_name]}"

        orbit_period_days = random.randint(50, 1000)

        x0 = round(base["x"] + random.uniform(-10, 10), 2)
        y0 = round(base["y"] + random.uniform(-10, 10), 2)
        z0 = round(base["z"] + random.uniform(-10, 10), 2)

        planets.append({
            "planet_id": planet_id_counter,
            "planet": name,
            "system_id": system_id,
            "orbit_period_days": orbit_period_days,
            "day0_x_coord": x0,
            "day0_y_coord": y0,
            "day0_z_coord": z0
        })
        planet_id_counter += 1

# === DataFrames ===
df_galaxies = pd.DataFrame(galaxies)
df_systems = pd.DataFrame(systems)
df_planets = pd.DataFrame(planets)

# === Podgląd danych ===
print("Galaxies:")
print(df_galaxies.head(), "\n")

print("Systems:")
print(df_systems.head(), "\n")

print("Planets:")
print(df_planets.head(), "\n")

# Eksport (opcjonalny)
df_galaxies.to_csv("galaxy_final.csv", index=False)
df_systems.to_csv("planetary_system_final.csv", index=False)
df_planets.to_csv("planet_final.csv", index=False)


Galaxies:
   galaxy_id           galaxy
0          1    Polaris Nexus
1          2      Kronis Ring
2          3      Omega Verge
3          4  Virelia Cluster
4          5     Zerion Reach 

Systems:
   system_id    system   central_star  x_coord  y_coord  z_coord  galaxy_id
0          1     Ixion     Ixion Star     0.00     0.00     0.00          1
1          2     Trion     Trion Star   382.99  -376.45  -748.79          1
2          3  Neptunus  Neptunus Star   376.79  -377.99  -785.80          1
3          4    Axioma    Axioma Star   413.90  -531.12  -732.66          1
4          5    Aether    Aether Star   318.33   -98.51  -316.46          2 

Planets:
   planet_id    planet  system_id  orbit_period_days  day0_x_coord  \
0          1      Rhea          1                754         -1.21   
1          2  Cerestra          1                547         -8.78   
2          3     Notos          2                713        377.42   
3          4  Eucharis          2                208

In [3]:
def generate_features():
    x = max(0, int(np.floor(np.random.normal(3, 2))))
    indices = random.sample(range(1, len(special_features_base) + 1), min(x, len(special_features_base)))
    return indices


special_features_base = [
    "pink_sun", "comets_rain", "emerald_lakes", "diamond_rain", "1000*C",
    "huge_gravity", "sulfuric_acid_fog", "magnetic_storms", "glowing_mountains",
    "eternal_night", "lava_rivers","purple_vegetation","bioluminescent_forests", 
    "giant_crystal_fields","frozen_oceans","constant_lightning","mirror_oceans"
]
planet_feature = []

for planet_id in [planet["planet_id"] for planet in planets]:
    features_id = generate_features()
    for feature_id in features_id:
        planet_feature.append({
            'planet_id': planet_id,
            'feature_id': feature_id
        })
planet_feature = pd.DataFrame(planet_feature)
planet_feature.to_csv("planet_feature_final.csv", index=False)

In [4]:
special_feature = [
    {'feature_id': i + 1, 'feature': feature}
    for i, feature in enumerate(special_features_base)
]

special_feature = pd.DataFrame(special_feature)
special_feature.to_csv('special_feature_final.csv', index = False)

In [5]:
df_planets

,planet_id,planet,system_id,orbit_period_days,day0_x_coord,day0_y_coord,day0_z_coord
0,1,Rhea,1,754,-1.21,-0.11,-6.61
1,2,Cerestra,1,547,-8.78,3.39,-5.62
2,3,Notos,2,713,377.42,-377.68,-751.52
3,4,Eucharis,2,208,378.62,-373.66,-740.42
4,5,Thyone,2,807,375.32,-380.28,-757.73
5,6,Euryale,2,391,386.44,-380.88,-741.45
6,7,Lemuria,3,939,367.47,-368.01,-792.49
7,8,Caelus,3,929,370.29,-373.08,-789.96
8,9,Astraeus,4,317,413.00,-528.06,-730.11
9,10,Mnestra,4,749,414.80,-534.60,-738.30


In [6]:
df_systems

,system_id,system,central_star,x_coord,y_coord,z_coord,galaxy_id
0,1,Ixion,Ixion Star,0.00,0.00,0.00,1
1,2,Trion,Trion Star,382.99,-376.45,-748.79,1
2,3,Neptunus,Neptunus Star,376.79,-377.99,-785.80,1
3,4,Axioma,Axioma Star,413.90,-531.12,-732.66,1
4,5,Aether,Aether Star,318.33,-98.51,-316.46,2
5,6,Ilyssar,Ilyssar Star,317.68,61.72,-378.65,2
6,7,Phantos,Phantos Star,778.25,-416.72,492.12,3
7,8,Aether,Aether Star,927.30,-449.29,480.44,3
8,9,Zantheos,Zantheos Star,308.97,92.54,-348.82,4
9,10,Liora,Liora Star,212.91,160.80,-355.74,4


In [7]:
df_galaxies

,galaxy_id,galaxy
0,1,Polaris Nexus
1,2,Kronis Ring
2,3,Omega Verge
3,4,Virelia Cluster
4,5,Zerion Reach


trip types

In [8]:
# Zakładam, że df_planets już istnieje (jeśli nie, trzeba go wcześniej załadować z CSV)

# Step 1: Generate a trip type for each planet
trip_type = []

for i, row in df_planets.iterrows():
    distance_from_center = math.sqrt(row["day0_x_coord"]**2 + row["day0_y_coord"]**2 + row["day0_z_coord"]**2)
    p = distance_from_center/math.sqrt(1110**2*3)
    is_round_trip = np.random.binomial(1, p) == 0
    
    trip_type.append({
        "trip_type_id": i + 1,
        "name": f"Mission to {row['planet']}",
        "coordinates": f"X:{row['day0_x_coord']}, Y:{row['day0_y_coord']}, Z:{row['day0_z_coord']}",
        "is_round_trip": is_round_trip,
        "planet_id": row["planet_id"],
        "orbital_period_days":row["orbit_period_days"] 
    })

# Convert to DataFrame and save
df_trip_type = pd.DataFrame(trip_type)
df_trip_type_final = df_trip_type.drop(columns=["orbital_period_days", "coordinates"])

# Zmień ścieżkę na swoją lokalną, np. na Pulpit
df_trip_type_final.to_csv("trip_type_final.csv", index=False)

# Show previe


In [9]:
df_trip_type_final

,trip_type_id,name,is_round_trip,planet_id
0,1,Mission to Rhea,True,1
1,2,Mission to Cerestra,True,2
2,3,Mission to Notos,False,3
3,4,Mission to Eucharis,True,4
4,5,Mission to Thyone,False,5
5,6,Mission to Euryale,True,6
6,7,Mission to Lemuria,False,7
7,8,Mission to Caelus,False,8
8,9,Mission to Astraeus,False,9
9,10,Mission to Mnestra,True,10


employees

In [10]:


fake = Faker()

positions = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
position_weights = [1, 1, 1, 4, 4]

salary_ranges = {
    "Technician": (4000, 6000),
    "Mission Specialist": (8000, 11000),
    "Commander": (9000, 12000),
    "Pilot": (6000, 8500),
    "Navigator": (5500, 8000)
}

# Parametry czasu
start_date = datetime(2023, 1, 1)
end_random_hire = datetime(2025, 6, 1)
today = datetime.today()

employees_data = []
num_employees = 60
pct_early_hires = 0.75
early_hires = int(num_employees * pct_early_hires)

for i in range(num_employees):
    first_name = fake.first_name()
    last_name = fake.last_name()

    position = random.choices(positions, weights=position_weights, k=1)[0]
    low, high = salary_ranges[position]

    # Hire date
    if i < early_hires:
        hire_date = start_date
    else:
        rand_days = random.randint(0, (end_random_hire - start_date).days)
        hire_date = start_date + timedelta(days=rand_days)

    # Staż w latach (pełne lata)
    years_worked = max(0, (today - hire_date).days // 365)

    # Wynagrodzenie bazowe + podwyżki
    base_salary = random.randrange(low, high + 1, 100)
    adjusted_salary = round(base_salary * (1.07 ** years_worked), -2)  # zaokrąglenie do setek

    employees_data.append({
        "employee_id": i + 1,
        "first_name": first_name,
        "last_name": last_name,
        "position": position,
        "hire_date": hire_date.strftime("%Y-%m-%d"),
        "fire_date": None,
        "salary": adjusted_salary
    })

# DataFrame
df_employees = pd.DataFrame(employees_data)
df_employees.to_csv("employee_final.csv", index=False)



trips+rocket

In [11]:
def is_employee_available(emp_id, new_start, new_end):
    
    employee = next((e for e in employees_data if e["employee_id"] == emp_id), None)
    if not employee:
        return False 
    
    hire_date = datetime.strptime(employee["hire_date"], "%Y-%m-%d")
    if new_start < hire_date:
        return False
    
    if employee["fire_date"]:
        fire_date = datetime.strptime(employee["fire_date"], "%Y-%m-%d")
        if new_end > fire_date:
            return False
    
    for trip in trip_employees:
        if trip["employee_id"] == emp_id:
            # znajdź zakres dat tej misji
            trip_id = trip["trip_id"]
            trip_info = next((t for t in trips if t["trip_id"] == trip_id), None)
            if trip_info:
                existing_start = datetime.strptime(trip_info["departure_date"], "%Y-%m-%d")
                existing_end = datetime.strptime(trip_info["return_date"], "%Y-%m-%d")
                # sprawdź konflikt
                if not (new_end < existing_start or new_start > existing_end):
                    return False
    return True


In [12]:
def pozycja_planety_w_czasie(x0, y0, z0, center, okres_dni, departure_dt):
    R = math.sqrt((x0 - center[0])**2 + (y0 - center[1])**2 + (z0 - center[2])**2)
    v0 = [x0 - center[0], y0 - center[1], z0 - center[2]]
    u = [x / math.sqrt(sum(i**2 for i in v0)) for x in v0]
    pomocniczy = [0, 1, 0] if abs(u[0]) < 1e-6 and abs(u[1]) < 1e-6 else [0, 0, 1]

    v_temp = [
        u[1]*pomocniczy[2] - u[2]*pomocniczy[1],
        u[2]*pomocniczy[0] - u[0]*pomocniczy[2],
        u[0]*pomocniczy[1] - u[1]*pomocniczy[0]
    ]
    v_magnitude = math.sqrt(sum(i**2 for i in v_temp))
    v = [i / v_magnitude for i in v_temp]

    omega = 2 * math.pi / (okres_dni * 24)  
    t = (departure_dt - datetime(2023, 1, 1)).total_seconds() / 3600 
    theta = omega * t
    cos_t, sin_t = math.cos(theta), math.sin(theta)
    pos = [
        center[0] + R * cos_t * u[0] + R * sin_t * v[0],
        center[1] + R * cos_t * u[1] + R * sin_t * v[1],
        center[2] + R * cos_t * u[2] + R * sin_t * v[2],
    ]
    return pos


In [13]:
def oblicz_cel_lotu_3d(
    start_rakiety,
    predkosc_rakiety_kmh,
    jednostka_km,
    srodek_orbity,
    pozycja_planety_start,
    okres_orbitalny_dni,
    epsilon=1e-5,
    max_iter=100000
):
    def dystans3d(p1, p2):
        return math.sqrt(
            (p1[0] - p2[0])**2 +
            (p1[1] - p2[1])**2 +
            (p1[2] - p2[2])**2
        )

    def wektor_sub(a, b):
        return [a[i] - b[i] for i in range(3)]

    def wektor_add(a, b):
        return [a[i] + b[i] for i in range(3)]

    def skalar_krotka(k, v):
        return [k * x for x in v]

    def normalizuj(v):
        d = math.sqrt(sum(x**2 for x in v))
        return [x / d for x in v]

    def cross(a, b):
        return [
            a[1]*b[2] - a[2]*b[1],
            a[2]*b[0] - a[0]*b[2],
            a[0]*b[1] - a[1]*b[0]
        ]

    # --- Przygotowanie orbity w przestrzeni 3D ---
    R = dystans3d(pozycja_planety_start, srodek_orbity)
    v0 = wektor_sub(pozycja_planety_start, srodek_orbity)
    u = normalizuj(v0)

    # Znajdź drugi wektor ortonormalny
    if abs(u[0]) < 1e-6 and abs(u[1]) < 1e-6:
        pomocniczy = [0, 1, 0]
    else:
        pomocniczy = [0, 0, 1]

    v = normalizuj(cross(u, pomocniczy))
    w = cross(v, u)  # finalne u, v – ortonormalna baza płaszczyzny orbity

    # Kąt początkowy (theta = 0 to pozycja startowa)
    theta0 = 0
    omega = 2 * math.pi / (okres_orbitalny_dni * 24)  # rad/h

    def pozycja_planety(t):
        theta = theta0 + omega * t
        cos_t = math.cos(theta)
        sin_t = math.sin(theta)
        pos = wektor_add(
            srodek_orbity,
            skalar_krotka(R * cos_t, u)
        )
        pos = wektor_add(
            pos,
            skalar_krotka(R * sin_t, v)
        )
        return tuple(pos)

    # Szacowanie czasu
    dyst_do_srodka = dystans3d(start_rakiety, srodek_orbity)
    d_min = max(0, dyst_do_srodka - R)
    d_max = dyst_do_srodka + R

    t_min = (d_min * jednostka_km) / predkosc_rakiety_kmh
    t_max = (d_max * jednostka_km) / predkosc_rakiety_kmh

    najlepszy_t = None
    najmniejszy_blad = float('inf')

    for _ in range(max_iter):
        t_mid = (t_min + t_max) / 2
        pos_planety = pozycja_planety(t_mid)

        dyst = dystans3d(start_rakiety, pos_planety) * jednostka_km
        t_lot = dyst / predkosc_rakiety_kmh

        blad = abs(t_lot - t_mid)

        if blad < najmniejszy_blad:
            najmniejszy_blad = blad
            najlepszy_t = t_mid

        if blad < epsilon:
            return t_mid, pos_planety

        if t_lot > t_mid:
            t_min = t_mid
        else:
            t_max = t_mid

    print(f" Osiągnięto limit iteracji ({max_iter}), zwracam najlepsze przybliżenie.")
    return najlepszy_t, pozycja_planety(najlepszy_t)


tu chyba za malo opracownikow

In [14]:
rockets = ['Stellar Vortex', 'Nebula Strider', 'Quantum Lance', 'Solaris Ark']
employees_roles = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
start_date = datetime(2023, 1, 1)
trips = []
trip_employees = []
k = 0
one_way_retired_employees = set()
last_id = employees_data[-1]["employee_id"]

rocket_data = []
rocket_names_prefixes = ["Stellar", "Nebula", "Quantum", "Solaris", "Lunar", "Cosmic", "Eclipse", "Nova", "Astro", "Galactic"]
rocket_names_suffixes = ["Vortex", "Strider", "Lance", "Ark", "Drift", "Fury", "Burst", "Wave", "Wraith", "Beacon"]
rocket_id_counter = 1
for prefix in rocket_names_prefixes:
    for suffix in random.sample(rocket_names_suffixes, 2):  # 2 kombinacje dla każdego prefixu
        rocket_data.append({
            "rocket_id": rocket_id_counter,
            "name": f"{prefix} {suffix}",
            "capacity": random.randint(4, 12)
        })
        rocket_id_counter += 1

df_rockets = pd.DataFrame(rocket_data)

# Inicjalizacja harmonogramu dla każdej rakiety (lista zajętości czasowych)
rocket_schedule = {r["rocket_id"]: [] for r in rocket_data}

for i, row in tqdm(df_trip_type.iterrows(), total=len(df_trip_type), desc="Generowanie misji"):
    num_variants = random.randint(1, 5)
    for _ in range(num_variants):
        trip_id = len(trips) + 1
        departure = start_date + timedelta(days=random.randint(0, 600))
        duration = random.randint(5, 30)
        speed = random.uniform(1000 * 60 * 60, 300000 * 60 * 60)
        rocket_id = random.randint(1, len(rockets))

        # PLANETA STARTOWA: 4
        planeta_4 = df_planets[df_planets["planet_id"] == 4].iloc[0]
        system_4 = df_systems[df_systems["system_id"] == planeta_4["system_id"]].iloc[0]
        start_rakiety = pozycja_planety_w_czasie(
            planeta_4["day0_x_coord"], planeta_4["day0_y_coord"], planeta_4["day0_z_coord"],
            [system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
            planeta_4["orbit_period_days"], departure
        )

        # CEL
        planet_id = row["planet_id"]
        planet_row = df_planets[df_planets["planet_id"] == planet_id].iloc[0]
        system_row = df_systems[df_systems["system_id"] == planet_row["system_id"]].iloc[0]
        srodek_orbity = [system_row["x_coord"], system_row["y_coord"], system_row["z_coord"]]
        okres_orbitalny_dni = planet_row["orbit_period_days"]
        pozycja_planety_start = pozycja_planety_w_czasie(
            planet_row["day0_x_coord"], planet_row["day0_y_coord"], planet_row["day0_z_coord"],
            srodek_orbity, okres_orbitalny_dni, departure
        )

        time_hours, _ = oblicz_cel_lotu_3d(
            start_rakiety=start_rakiety,
            predkosc_rakiety_kmh=speed,
            jednostka_km=50000,
            srodek_orbity=srodek_orbity,
            pozycja_planety_start=pozycja_planety_start,
            okres_orbitalny_dni=okres_orbitalny_dni
        )
        time_days_1 = time_hours / 24

        is_round_trip = row["is_round_trip"]
        if not is_round_trip:
            return_date = datetime(9999, 1, 1)
        else:
            pozycja_planety_powrot = pozycja_planety_w_czasie(
                planet_row["day0_x_coord"], planet_row["day0_y_coord"], planet_row["day0_z_coord"],
                srodek_orbity, okres_orbitalny_dni,
                departure + timedelta(days=time_days_1 + duration)
            )
            pozycja_4_planety = pozycja_planety_w_czasie(
                planeta_4["day0_x_coord"], planeta_4["day0_y_coord"], planeta_4["day0_z_coord"],
                [system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
                planeta_4["orbit_period_days"],
                departure + timedelta(days=time_days_1 + duration)
            )
            time_hours, _ = oblicz_cel_lotu_3d(
                start_rakiety=pozycja_planety_powrot,
                predkosc_rakiety_kmh=speed,
                jednostka_km=50000,
                srodek_orbity=[system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
                pozycja_planety_start=pozycja_4_planety,
                okres_orbitalny_dni=planeta_4["orbit_period_days"]
            )
            time_days_2 = time_hours / 24
            total_days = time_days_1 + time_days_2 + duration
            return_date = departure + timedelta(days=total_days)

        # Wybór dostępnej rakiety (niezajętej w tym czasie)
        available_rockets = []
        for rocket_id, schedule in rocket_schedule.items():
            overlaps = any(
                not (return_date <= trip["departure"] or departure >= trip["return"])
                for trip in schedule
            )
            if not overlaps:
                available_rockets.append(rocket_id)

        if not available_rockets:
            # Nie ma dostępnej rakiety, pomiń generowanie tej misji
            continue

        rocket_id = random.choice(available_rockets)

        # Zarejestruj czas zajętości rakiety
        rocket_schedule[rocket_id].append({
            "departure": departure,
            "return": return_date
        })
        # Dodaj losowe opóźnienie (w dniach)
        delay_days = random.choice([0]*8 + [np.floor(np.random.normal(0, 5))])

        trips.append({
    "trip_id": trip_id,
    "trip_type_id": row["trip_type_id"],
    "rocket_id": rocket_id,
    "departure_date": departure.strftime("%Y-%m-%d"),
    "return_date": return_date.strftime("%Y-%m-%d"),
    "delay_days": delay_days,
    "status": "Delayed" if delay_days > 0 else random.choice(["Scheduled", "Completed"]),
    "speed": speed
})


        # Przypisanie pracowników
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0
        assigned = []

        for role in min_roles:
            available = [e for e in employees_data if e["position"] == role and
                        is_employee_available(e["employee_id"], departure, return_date) and
                        e["employee_id"] not in one_way_retired_employees]
            
            if available:
                emp = random.choice(available)
            else:
                # Zatrudnienie nowego pracownika na brakującą pozycję
                last_id += 1
                emp = {
                    "employee_id": last_id,
                    "first_name": fake.first_name(),
                    "last_name": fake.last_name(),
                    "position": role,
                    "hire_date": departure.strftime("%Y-%m-%d"),
                    "fire_date": None,
                    "salary": random.randint(6000, 9000)
                }
                employees_data.append(emp)

            assigned.append(emp["employee_id"])
            k += 1
            trip_employees.append({
                "trip_employee_id": k,
                "role_in_trip": emp["position"],
                "employee_id": emp["employee_id"],
                "trip_id": trip_id
            })

            if not is_round_trip:
                one_way_retired_employees.add(emp["employee_id"])
                
                for old_emp in employees_data:
                    if old_emp["employee_id"] == emp["employee_id"]:
                        old_emp["fire_date"] = departure.strftime("%Y-%m-%d")
                        break
                last_id += 1
                new_employee = {
                    "employee_id": last_id,
                    "first_name": fake.first_name(),
                    "last_name": fake.last_name(),
                    "position": emp["position"],
                    "hire_date": departure.strftime("%Y-%m-%d"),
                    "fire_date": None,
                    "salary": round(emp["salary"]*1.15, 2)
                }
                employees_data.append(new_employee)



        for _ in range(n_optional):
            available = [e for e in employees_data if e["position"] in other_roles and
                         is_employee_available(e["employee_id"], departure, return_date) and
                         e["employee_id"] not in one_way_retired_employees]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                k += 1
                trip_employees.append({
                    "trip_employee_id": k,
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })
                if not is_round_trip:
                    one_way_retired_employees.add(emp["employee_id"])

                    new_id = last_id + 1
                    new_employee = {
                        "employee_id": new_id,
                        "first_name": fake.first_name(),
                        "last_name": fake.last_name(),
                        "position": emp["position"],
                        "hire_date": departure.strftime("%Y-%m-%d"),
                        "salary": round(emp["salary"]*1.15, 2)
                    }
                    employees_data.append(new_employee)
                    last_id += 1

# Eksport

df_employees = pd.DataFrame(employees_data)
df_employees.to_csv("employee_final.csv", index=False)
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)
df_trips.to_csv("trip_final.csv", index=False)
df_trip_employees.to_csv("trip_employee_final.csv", index=False)
df_rockets.to_csv("rocket_final.csv", index=False)

Generowanie misji: 100%|██████████| 45/45 [00:01<00:00, 29.08it/s]


In [15]:


# rockets = ['Stellar Vortex', 'Nebula Strider', 'Quantum Lance', 'Solaris Ark']
# employees_roles = ["Technician", "Mission Specialist", "Commander", "Pilot", "Navigator"]
# start_date = datetime(2023, 1, 1)
# trips = []
# trip_employees = []
# k = 0
# one_way_retired_employees = set()
# last_id = employees_data[-1]["employee_id"]

# rocket_data = []
# rocket_names_prefixes = ["Stellar", "Nebula", "Quantum", "Solaris", "Lunar", "Cosmic", "Eclipse", "Nova", "Astro", "Galactic"]
# rocket_names_suffixes = ["Vortex", "Strider", "Lance", "Ark", "Drift", "Fury", "Burst", "Wave", "Wraith", "Beacon"]
# rocket_id_counter = 1
# for prefix in rocket_names_prefixes:
#     for suffix in random.sample(rocket_names_suffixes, 2):  # 2 kombinacje dla każdego prefixu
#         rocket_data.append({
#             "rocket_id": rocket_id_counter,
#             "name": f"{prefix} {suffix}",
#             "capacity": random.randint(4, 12)
#         })
#         rocket_id_counter += 1

# df_rockets = pd.DataFrame(rocket_data)

# # Inicjalizacja harmonogramu dla każdej rakiety (lista zajętości czasowych)
# rocket_schedule = {r["rocket_id"]: [] for r in rocket_data}

# for i, row in tqdm(df_trip_type.iterrows(), total=len(df_trip_type), desc="Generowanie misji"):
#     num_variants = random.randint(1, 5)
#     for _ in range(num_variants):
#         trip_id = len(trips) + 1
#         departure = start_date + timedelta(days=random.randint(0, 600))
#         duration = random.randint(5, 30)
#         speed = random.uniform(1000 * 60 * 60, 300000 * 60 * 60)
#         rocket_id = random.randint(1, len(rockets))

#         # PLANETA STARTOWA: 4
#         planeta_4 = df_planets[df_planets["planet_id"] == 4].iloc[0]
#         system_4 = df_systems[df_systems["system_id"] == planeta_4["system_id"]].iloc[0]
#         start_rakiety = pozycja_planety_w_czasie(
#             planeta_4["day0_x_coord"], planeta_4["day0_y_coord"], planeta_4["day0_z_coord"],
#             [system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
#             planeta_4["orbit_period_days"], departure
#         )

#         # CEL
#         planet_id = row["planet_id"]
#         planet_row = df_planets[df_planets["planet_id"] == planet_id].iloc[0]
#         system_row = df_systems[df_systems["system_id"] == planet_row["system_id"]].iloc[0]
#         srodek_orbity = [system_row["x_coord"], system_row["y_coord"], system_row["z_coord"]]
#         okres_orbitalny_dni = planet_row["orbit_period_days"]
#         pozycja_planety_start = pozycja_planety_w_czasie(
#             planet_row["day0_x_coord"], planet_row["day0_y_coord"], planet_row["day0_z_coord"],
#             srodek_orbity, okres_orbitalny_dni, departure
#         )

#         time_hours, _ = oblicz_cel_lotu_3d(
#             start_rakiety=start_rakiety,
#             predkosc_rakiety_kmh=speed,
#             jednostka_km=50000,
#             srodek_orbity=srodek_orbity,
#             pozycja_planety_start=pozycja_planety_start,
#             okres_orbitalny_dni=okres_orbitalny_dni
#         )
#         time_days_1 = time_hours / 24

#         is_round_trip = row["is_round_trip"]
#         if not is_round_trip:
#             return_date = datetime(9999, 1, 1)
#         else:
#             pozycja_planety_powrot = pozycja_planety_w_czasie(
#                 planet_row["day0_x_coord"], planet_row["day0_y_coord"], planet_row["day0_z_coord"],
#                 srodek_orbity, okres_orbitalny_dni,
#                 departure + timedelta(days=time_days_1 + duration)
#             )
#             pozycja_4_planety = pozycja_planety_w_czasie(
#                 planeta_4["day0_x_coord"], planeta_4["day0_y_coord"], planeta_4["day0_z_coord"],
#                 [system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
#                 planeta_4["orbit_period_days"],
#                 departure + timedelta(days=time_days_1 + duration)
#             )
#             time_hours, _ = oblicz_cel_lotu_3d(
#                 start_rakiety=pozycja_planety_powrot,
#                 predkosc_rakiety_kmh=speed,
#                 jednostka_km=50000,
#                 srodek_orbity=[system_4["x_coord"], system_4["y_coord"], system_4["z_coord"]],
#                 pozycja_planety_start=pozycja_4_planety,
#                 okres_orbitalny_dni=planeta_4["orbit_period_days"]
#             )
#             time_days_2 = time_hours / 24
#             total_days = time_days_1 + time_days_2 + duration
#             return_date = departure + timedelta(days=total_days)

#         # Wybór dostępnej rakiety (niezajętej w tym czasie)
#         available_rockets = []
#         for rocket_id, schedule in rocket_schedule.items():
#             overlaps = any(
#                 not (return_date <= trip["departure"] or departure >= trip["return"])
#                 for trip in schedule
#             )
#             if not overlaps:
#                 available_rockets.append(rocket_id)

#         if not available_rockets:
#             # Nie ma dostępnej rakiety, pomiń generowanie tej misji
#             continue

#         rocket_id = random.choice(available_rockets)

#         # Zarejestruj czas zajętości rakiety
#         rocket_schedule[rocket_id].append({
#             "departure": departure,
#             "return": return_date
#         })
#         # Dodaj losowe opóźnienie (w dniach)
#         delay_days = random.choice([0]*8 + [np.floor(np.random.normal(0, 5))])

#         trips.append({
#     "trip_id": trip_id,
#     "trip_type_id": row["trip_type_id"],
#     "rocket_id": rocket_id,
#     "departure_date": departure.strftime("%Y-%m-%d"),
#     "return_date": return_date.strftime("%Y-%m-%d"),
#     "delay_days": delay_days,
#     "status": "Delayed" if delay_days > 0 else random.choice(["Scheduled", "Completed"]),
#     "speed": speed
# })


#         # Przypisanie pracowników
#         min_roles = ["Pilot", "Navigator"]
#         other_roles = ["Technician", "Mission Specialist", "Commander"]
#         n_optional = random.randint(1, 2) if is_round_trip else 0
#         assigned = []

#         for role in min_roles:
#             available = [e for e in employees_data if e["position"] == role and
#                          is_employee_available(e["employee_id"], departure, return_date) and
#                          e["employee_id"] not in one_way_retired_employees]
            
#             #if not available:
#                 #raise ValueError(f"No available employee for role: {role} (without conflict)")
#             if not available:
#                 print(f"[INFO] Brak dostępnych pracowników dla roli {role} — pomijam tę podróż")
#                 continue  # pomiń przydzielanie lub całą iterację

#             emp = random.choice(available)
#             assigned.append(emp["employee_id"])
#             k += 1
#             trip_employees.append({
#                 "trip_employee_id": k,
#                 "role_in_trip": emp["position"],
#                 "employee_id": emp["employee_id"],
#                 "trip_id": trip_id
#             })
#             if not is_round_trip:
#                 one_way_retired_employees.add(emp["employee_id"])
                
                
#                 # Ustaw fire_date dla starego pracownika
#                 for old_emp in employees_data:
#                     if old_emp["employee_id"] == emp["employee_id"]:
#                         old_emp["fire_date"] = departure.strftime("%Y-%m-%d")
#                         break
#                 new_id = last_id + 1
#                 new_employee = {
#                     "employee_id": new_id,
#                     "first_name": fake.first_name(),
#                     "last_name": fake.last_name(),
#                     "position": emp["position"],
#                     "hire_date": departure.strftime("%Y-%m-%d"),
#                     "fire_date": None,
#                     "salary": round(emp["salary"]*1.15, 2)
#                 }
#                 employees_data.append(new_employee)
#                 last_id += 1


#         for _ in range(n_optional):
#             available = [e for e in employees_data if e["position"] in other_roles and
#                          is_employee_available(e["employee_id"], departure, return_date) and
#                          e["employee_id"] not in one_way_retired_employees]
#             if available:
#                 emp = random.choice(available)
#                 assigned.append(emp["employee_id"])
#                 k += 1
#                 trip_employees.append({
#                     "trip_employee_id": k,
#                     "trip_id": trip_id,
#                     "employee_id": emp["employee_id"],
#                     "role_in_trip": emp["position"]
#                 })
#                 if not is_round_trip:
#                     one_way_retired_employees.add(emp["employee_id"])

#                     new_id = last_id + 1
#                     new_employee = {
#                         "employee_id": new_id,
#                         "first_name": fake.first_name(),
#                         "last_name": fake.last_name(),
#                         "position": emp["position"],
#                         "hire_date": departure.strftime("%Y-%m-%d"),
#                         "salary": round(emp["salary"]*1.15, 2)
#                     }
#                     employees_data.append(new_employee)
#                     last_id += 1

# # Eksport

# df_employees = pd.DataFrame(employees_data)
# df_employees.to_csv("employee_final.csv", index=False)
# df_trips = pd.DataFrame(trips)
# df_trip_employees = pd.DataFrame(trip_employees)
# df_trips.to_csv("trip_final.csv", index=False)
# df_trip_employees.to_csv("trip_employee_final.csv", index=False)
# df_rockets.to_csv("rocket_final.csv", index=False)

In [16]:
df_employees

,employee_id,first_name,last_name,position,hire_date,fire_date,salary
0,1,Samuel,Porter,Navigator,2023-01-01,2024-04-25,8000.00
1,2,Charles,Franco,Navigator,2023-01-01,2023-09-01,8600.00
2,3,Linda,Hill,Technician,2023-01-01,None,4700.00
3,4,Jessica,Rodriguez,Technician,2023-01-01,None,6100.00
4,5,Emily,Jones,Pilot,2023-01-01,2023-03-30,8600.00
...,...,...,...,...,...,...,...
93,94,Mitchell,Ramirez,Navigator,2024-06-23,None,11902.50
94,95,Nicole,Gray,Pilot,2024-08-03,None,9522.00
95,96,Julie,Gomez,Navigator,2024-08-03,None,11770.25
96,97,Travis,Fitzpatrick,Navigator,2023-11-13,None,7551.00


transaction + booking

In [17]:
def distance_to_cost(trip_id):
    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    start_time = datetime.strptime(trip_row["departure_date"], "%Y-%m-%d")
    end_time = datetime.strptime(trip_row["return_date"], "%Y-%m-%d")
    fly_time =  end_time - start_time 
    if end_time == datetime(9999, 1, 1):
        fly_time = timedelta(days=100)
    speed = trip_row["speed"]

    distance = fly_time.total_seconds() / 3600 * speed
    return distance

In [20]:
fake = Faker()

n = 1000

# Step 1: Generate clients
clients = [{
    "client_id": i + 1,
    "first_name": fake.first_name(),
    "last_name": fake.last_name(),
    "client_contact_id": i + 1,
    "emergency_contact_id": i + 1
} for i in range(n)]
df_clients = pd.DataFrame(clients)

emergency_contact = [{
    "emergency_contact_id": i + 1,
    "phone": fake.phone_number(),
    "email": fake.email()
} for i in range(n)]
df_emergency_contact = pd.DataFrame(emergency_contact)

client_contact = [{
    "client_contact_id": i + 1,
    "phone": fake.phone_number(),
    "email": fake.email()
} for i in range(n)]
df_client_contact = pd.DataFrame(client_contact)

# Step 2: Generate bookings
bookings = []
for client in clients:
    n_bookings = random.randint(1, 3)
    booked_trip_ids = random.sample(df_trips["trip_id"].tolist(), n_bookings)
    for trip_id in booked_trip_ids:
        bookings.append({
            "booking_id": len(bookings) + 1,
            "seat_number": random.randint(1, 12),
            "client_id": client["client_id"],
            "trip_id": trip_id
        })
df_bookings = pd.DataFrame(bookings)

# Pre-map data
planet_coords = df_trip_type.set_index("trip_type_id")["coordinates"].to_dict()
round_trip_flags = df_trip_type.set_index("trip_type_id")["is_round_trip"].to_dict()
trip_types_planet = df_trip_type.set_index("trip_type_id")["planet_id"].to_dict()
rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

booking_seat_counts = df_bookings.groupby(["client_id", "trip_id"]).size().to_dict()

transactions = []

# Cache start rakiety z planety 4
planeta_4 = df_planets[df_planets["planet_id"] == 4].iloc[0]
system_4 = df_systems[df_systems["system_id"] == planeta_4["system_id"]].iloc[0]


# Pasek postępu
for booking in tqdm(bookings, desc="Przetwarzanie transakcji"):
    trip_id = booking["trip_id"]
    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    trip_type_id = trip_row["trip_type_id"]
    rocket_id = trip_row["rocket_id"]
    departure = datetime.strptime(trip_row["departure_date"], "%Y-%m-%d")

    # OBLICZ dystans
    distance = distance_to_cost(booking["trip_id"])

    # CENA
    employee_count = df_trip_employees[df_trip_employees["trip_id"] == trip_id].shape[0]
    key = (booking["client_id"], trip_id)
    num_seats = booking_seat_counts.get(key, 1)

    base_price = 10000
    base_price += num_seats * 200
    base_price += (distance/10**6)

    base_price += rockets_capacity[rocket_id] * 100
    base_price += employee_count * 150
    if not round_trip_flags[trip_type_id]:
        base_price *= 2


    # Sprawdź duplikaty
    if key not in {(t["client_id"], t["trip_id"]) for t in transactions}:
        transactions.append({
            "transaction_id": len(transactions) + 1,
            "amount": round(base_price, 2),
            "method": random.choice(["card", "transfer", "crypto"]),
            "payment_date": fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
            "trip_id": trip_id,
            "client_id": booking["client_id"],
            "num_seats": num_seats
        })

# Finalna tabela
df_transactions = pd.DataFrame(transactions)

# Zapisz dane
df_clients.to_csv("client_final.csv", index=False)

df_client_contact.to_csv("client_contact_final.csv", index=False)

df_emergency_contact.to_csv("emergency_contact_final.csv", index=False)

df_booking_final=df_bookings.drop(columns=["seat_number"])
df_booking_final.to_csv("booking_final.csv", index=False)

df_transactions_final=df_transactions.drop(columns=["num_seats"])
df_transactions_final.to_csv("transaction_final.csv", index=False)

Przetwarzanie transakcji: 100%|██████████| 2041/2041 [00:05<00:00, 344.45it/s]


cost - na razie generowane wyzej jest

In [21]:
costs = []
employee_counts = df_trip_employees.groupby("trip_id").size().to_dict()
income_per_trip = df_transactions.groupby("trip_id")["amount"].sum().to_dict()

# Definicje udziałów kosztów wg kategorii (bardziej realistyczne)
cost_categories = {
    "Fuel": 0.4,
    "Crew": 0.25,
    "Maintenance": 0.2,
    "Logistics": 0.15
}

for trip in df_trips.itertuples():
    trip_id = trip.trip_id
    trip_type_id = trip.trip_type_id
    distance = distance_to_cost(trip_id)
    is_round = round_trip_flags[trip_type_id]
    n_employees = employee_counts.get(trip_id, 1)
    max_income = income_per_trip.get(trip_id, 10000)

    # Bazowy koszt — zależny od liczby pracowników i dystansu
    base_cost = 7000 + (n_employees * 5000) + (distance / 1e6) * 0.8
    if not is_round:
        base_cost *= 2

    # Finalny koszt < 90% dochodu
    total_cost = min(base_cost, max_income * random.uniform(0.7, 0.9))

    for category, share in cost_categories.items():
        amount = round(total_cost * share, 2)
        costs.append({
            "cost_id": len(costs) + 1,
            "trip_id": trip_id,
            "description": category,
            "amount": amount
        })

df_costs = pd.DataFrame(costs)
df_costs.to_csv("cost_final.csv", index=False)


clients + adress + emergency_contact